In [5]:
import pandas as pd
from numpy import dtype
from ludwig.api import LudwigModel
from new_api import (add_entity, create_new_entity_set, add_aggregation_features, get_training_df, get_prediction_df,
add_entity_df, list_features, get_training_df_el, generate_base_model_definition)


In [6]:
df = pd.read_csv('titanic.csv')

passenger_entity = {"name": "passengers", "table": "passengers", "type": "primary", "index": "PassengerId"}

add_entity_df(df, passenger_entity)

create_new_entity_set(name="titanic", entities=["passengers"], relationships=[])

eol_df = df[["PassengerId", "Survived"]]


eol = {"pk": "PassengerId", "label": "Survived"}

features = {"entity_set": "titanic",
            "target_entity": "passengers",
            "features": {
                "passengers": ["Pclass","Sex","Age","SibSp","Parch","Ticket","Fare","Cabin","Embarked"]


            },
            "observations": {"type": "el", "eol": eol, "data": eol_df}

            }
training_df = get_training_df_el(features)
training_df


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,0,373450,8.0500,None,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,211536,13.0000,None,S
887,888,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,female,NaN,1,2,W./C. 6607,23.4500,None,S
889,890,1,1,male,26.0,0,0,111369,30.0000,C148,C


In [7]:
generate_base_model_definition(features)

In [ ]:
# %load model_definition.yaml
input_features:
- name: PassengerId
  type: numerical
- name: Pclass
  type: numerical
- name: Sex
  type: category
- name: Age
  type: numerical
- name: SibSp
  type: numerical
- name: Parch
  type: numerical
- name: Ticket
  type: category
- name: Fare
  type: numerical
- name: Cabin
  type: category
- name: Embarked
  type: category
output_features:
- name: Survived
  type: binary


## Iterate on model in cell below.  Results are written to a new directory every time

In [13]:
%%writefile model_definition.yaml
input_features:
- name: Pclass
  type: numerical
- name: Sex
  type: category
- name: Age
  type: numerical
  preprocessing:
          missing_value_strategy: fill_with_mean
          normalization: zscore
- name: SibSp
  type: numerical
- name: Parch
  type: numerical
- name: Fare
  type: numerical
- name: Cabin
  type: category
output_features:
- name: Survived
  type: binary
  num_fc_layers: 2
  fc_size: 128
training:
    batch_size: 128
    epochs: 200


Overwriting model_definition.yaml


In [14]:
ludwig_model = LudwigModel(model_definition_file='model_definition.yaml')
train_status = ludwig_model.train(data_df=df)

Instructions for updating:
targets is deprecated, use labels instead


In [15]:
max(train_status['validation']['combined']['accuracy'])

0.8024691358024691

In [ ]:
deploy_model_to_fastscore(ludwig_model)
#saves model to tmp directory (or locates experiement in results directory) and package as tar file
#generates a model conformance python file (same every time?) that just unpacks tar file
#using input features and output features generate schema definition file
#upload everything to fastscore using python sdk
#model_definition,yaml gets tagged and stored....now we can automate re-training model on new data and we can guarantee that model can be recreated.
